In [1]:
from mysql.connector import connect             # import connect module after check
import numpy  as np                             # numpy array manipulation
import pandas as pd                             # We wil be storing our data pulled from DB into dataframes for better management 
from pylab import rcParams                      # I have no idea what this is...
import pkgutil                                  # Just an optional import to check if a library exist in python..not very important
import traceback                                # For tracing errors exception thrown by interpreter
from string import Template                     # To make generic queries easier to change

ModuleNotFoundError: No module named 'mysql'

In [10]:
 ''' (1) if you wish to connect to the SRL database, enter your 'user' and 'password' in the _db_config variable below this comment. 
If you do not have user or password, contact <mathias.hiron@gmail.com> for gaining access '''
db_config = {
        'user'    : 'amine',
        'password': 'n1hFerXeQa2MRh7e',
        'host'    : 'franceioi.cinniket56wn.eu-central-1.rds.amazonaws.com',
        'database': 'srl',
        'port'    : '3306'
    }
    
    # if you wish to select tables you're interested in, uncomment what you need and comment what you don't need
tables_config    = [
        "participant",   # This table lists all participants 
        "participer",    # This table lists all participants who have participated into an activity
        "activite",      # This table lists all activities of a participant 
        "clavier",       # This table lists all keyboard logs, mainly used to track participant's inputs to check if user is idle or not
        "souris",        # This table lists all mouse logs, mainly used to track participant's mouse position to check if it has changed between two timestamp (Idle check)
        "modification",  # This table tracks all modifications that has been made by a participant in the code editor
        "navigation",    # This table tracks the participant's navigation through the platform (whether they are on the Help/Exercice/Home screen)
        "pas_a_pas",     # This table tracks the participant's usage of the step component in the platform
        "validation",    # This table lists all exercices done validated and/or experimented by a participant 
        "connexion",     # This table tracks all participants' connexions
        "sujet"          # This table lists all subjects done by learners
]

In [11]:
def check_module(module_name, package_name):
    if not pkgutil.find_loader(module_name):              # Check if a module is installed, used to not crash the execution and handle import stuff automatically
        eval("!pip install " + "mysql-connector-python")  # Install the connect module if not by providing the package 'mysql-connector-python' in argument

def get_connection(config):                     # We will pass into this procedure a configuration variable highlighting connection parameters (user, password, host, database, port)
    return connect(**config)                    # Open he connection and connect to the database
def close_connection(connection):               # Close connection after you're done.
    connection.close()

def get_tables_from_db(id_connection, table_config, SQL_query):
    df_tables = dict()
    mysql_connection = get_connection()
    df = dict()                                 # Data import 
    sql_connection = get_connection(db_config)

def query_db(query):
    df_table = {}
    mysql_connection = get_connection(db_config)
    with mysql_connection.cursor() as cursor: # while connected 
        try:
            cursor.execute(query)
            result = cursor.fetchall()
            columns_name = np.array([cursor.description[i][0] for i in range(len(cursor.description)) if cursor.description[i][0]])
            data = []
            for row in result:
                data.append(np.array([row[i] for i in range(len(row))]))
            df_table = pd.DataFrame(data, columns=columns_name)
        except Exception:
            traceback.print_exc()
    return df_table

In [12]:
# Query to retrieve all participant from 2022-05-31
def getParticipantMay():  
    query = """
    SELECT participant.id, participant.URI as participant_URI, participer.URI as participer_URI, timestamp_server as timestamp_MAI 
    FROM participer, participant, activite, connexion 
    WHERE participant.id=participer.id 
    AND participer.id=activite.id 
    AND timestamp REGEXP \"2022-05-31\" 
    ORDER BY timestamp DESC 
    """
    return query_db(query)

# Query to retrieve all participant from 2022-06-03
def getParticipantJune():   
    query = """
    SELECT participant.id, participant.URI as participant_URI, participer.URI as participer_URI, timestamp_server as timestamp_JUIN 
    FROM participer, participant, activite, connexion 
    WHERE participant.id=participer.id 
    AND participer.id=activite.id 
    AND timestamp REGEXP \"2022-06-03\" 
    ORDER BY timestamp DESC
    """
    return query_db(query)

# Query to DEBUG
def getParticipantMyself(): 
    query = """
    SELECT participant.id, participant.URI as participant_URI, participer.URI as participer_URI, timestamp_server 
    FROM participer, participant, activite, connexion 
    WHERE participant.id=participer.id 
    AND participer.id=activite.id 
    AND timestamp REGEXP \"2022-07-09\" 
    ORDER BY timestamp DESC
    """

def getParticipantExercices(learnerID=None, sujetURI="concours.castor-informatique.fr/"):
    temp = Template("""
    SELECT participant.id as LEARNER, sujet.URI as SUJET, version as VER, type_erreur as ERR, experimentation as EXP, score, validation.timestamp
    FROM participant, participer, connexion, validation, sujet
    WHERE participant.id=participer.id_participant
    AND participer.id=connexion.id_participation
    AND connexion.id=validation.id_connexion
    AND validation.id_sujet=sujet.id
    AND sujet.URI REGEXP "$sujet"
    GROUP BY participant.id, validation.timestamp
    ORDER BY validation.timestamp ASC
    """)
    temp2 = Template("""
    SELECT participant.id as LEARNER, sujet.URI as SUJET, version as VER, type_erreur as ERR, experimentation as EXP, score, validation.timestamp
    FROM participant, participer, connexion, validation, sujet
    WHERE participant.id=participer.id_participant
    AND participer.id=connexion.id_participation
    AND connexion.id=validation.id_connexion
    AND validation.id_sujet=sujet.id
    AND sujet.URI REGEXP "$sujet"
    AND participant.id=$participantID
    GROUP BY participant.id, validation.timestamp
    ORDER BY validation.timestamp ASC
    """)
    if learnerID==None:
        query = temp.substitute(sujet=sujetURI)
    else:
        query = temp2.substitute(participantID=learnerID, sujet=sujetURI)
    return query_db(query)

In [13]:
# Merge dataframes using participant URI
# res = pd.merge(getParticipantMay(), getParticipantJune(), how='inner', on=['participant_URI'])
particip_may       = getParticipantMay()
particip_june      = getParticipantJune()          

NameError: name 'connect' is not defined

In [6]:
particip_exercices = getParticipantExercices()       # Get query result in a dataFrame

NameError: name 'Template' is not defined

In [7]:
particip_exercices

NameError: name 'particip_exercices' is not defined

In [8]:
def getPossibleLearnerIDs(data):
    learnerIDs = set()
    dataSize = len(data)
    for i in range(dataSize):
        learnerIDs.add(data["LEARNER"][i])
    return learnerIDs

IDs = getPossibleLearnerIDs(data=particip_exercices) # Get all possible learnerIDs
IDs

NameError: name 'particip_exercices' is not defined

In [9]:
def updateDataColumnVal(data, columnName, updateFunc):
    dataSize = len(data)
    for i in range(dataSize):
        data[columnName][i] = updateFunc(data[columnName][i])
    return data

def getSubjectName(subjectURI):
    if subjectURI=="concours.castor-informatique.fr/1-melodie":
        return "Melodie"
    elif subjectURI=="concours.castor-informatique.fr/2-allers-retours-de-LEDs":
        return "Alternance"
    elif subjectURI=="concours.castor-informatique.fr/3-allers-retours-de-LEDs":
        return "Show Lumineux"
    elif subjectURI=="concours.castor-informatique.fr/4-afficher-la-direction":
        return "Quelle Direction"
    elif subjectURI=="concours.castor-informatique.fr/5-instrument-de-musique":
        return "Instrument"  # à vérifier dans la palteforme Quick-Pi
    elif subjectURI=="concours.castor-informatique.fr/6-light-show-2":
        return "Show Lumineux 2"
    elif subjectURI=="concours.castor-informatique.fr/7-avertisseur-de-recul":
        return "Avertisseur" # à vérifier dans la plateforme Quick-Pi
    elif subjectURI=="concours.castor-informatique.fr/8-servo-chronometre":
        return "Servo Chronometré"
particip_exercices=updateDataColumnVal(data=particip_exercices, 
                                       columnName="SUJET",
                                       updateFunc=getSubjectName)    

NameError: name 'particip_exercices' is not defined

In [12]:
particip_exercices

,LEARNER,SUJET,VER,ERR,EXP,score,timestamp
0,132,Alternance,2,Erreur de solution,0,0,2021-06-08 18:17:25
1,132,Alternance,2,Erreur de solution,0,0,2021-06-08 18:17:33
2,132,Alternance,2,Erreur de solution,0,0,2021-06-08 18:17:39
3,132,Alternance,2,Erreur de solution,0,0,2021-06-08 18:17:48
4,132,Alternance,2,Erreur de solution,0,0,2021-06-08 18:17:54
...,...,...,...,...,...,...,...
53049,4692,Avertisseur,2,Aucune,1,0,2021-07-22 16:31:09
53050,4692,Avertisseur,2,Erreur de solution,0,0,2021-07-22 16:32:29
53051,4692,Avertisseur,2,Erreur de solution,0,0,2021-07-22 16:34:25
53052,4692,Avertisseur,2,Erreur de solution,0,0,2021-07-22 16:35:25


In [13]:
#Filter dataFrame according to what you want (learnerID/version, etc.)
particip_exercices[  (particip_exercices.LEARNER==list(IDs)[3]) 
                   & (particip_exercices['SUJET'].str.contains('|'.join(["Melodie", "Alternance"])))
                  # & (particip_exercices.VER==2)
                  # & (particip_exercices.score==100)
                  ] 

,LEARNER,SUJET,VER,ERR,EXP,score,timestamp
103,163,Melodie,2,Aucune,1,0,2021-06-09 06:45:35
104,163,Melodie,2,Aucune,1,0,2021-06-09 06:45:36
105,163,Melodie,2,Aucune,1,0,2021-06-09 06:45:46
106,163,Melodie,2,Aucune,1,0,2021-06-09 06:45:52
107,163,Melodie,2,Aucune,1,0,2021-06-09 06:45:53
110,163,Alternance,2,Aucune,1,0,2021-06-09 06:47:00
111,163,Alternance,2,Aucune,1,0,2021-06-09 06:47:06
112,163,Alternance,2,Aucune,1,0,2021-06-09 06:47:08
114,163,Alternance,2,Aucune,1,0,2021-06-09 06:47:41
115,163,Alternance,2,Aucune,1,0,2021-06-09 06:47:53


In [14]:
"""
Build a Criteria list:
    + A learner must at least validate one or multiple exercices   (eg. requireValidation= True)
    + A learner must at least experiment one or multiple exercices (eg.requireExperiment=True)
    + A learner must at least validate an exercice of a specific version (eg. requireVersion=[2,3,4])
    + A learner must satisfies the criteria below on this exercises pool (eg. requireExercices=["Melodie","Alternance", ...])
"""
def selectLearners(data,
                  learnerIDs,
                  requireExercices=[],
                  requireVersion=[],
                  requireValidation=False,
                  requireExperiment=False,
                  ):   
    selectedLearners = []
    for learnerID in learnerIDs:
        # Checkers variables to see if criterias of selection are met
        exerciceCheck   = False
        versionCheck    = False
        validationCheck = False
        experimentCheck = False
        # Filter the data by the current learnerID of the for loop
        tempData = data[data.LEARNER==learnerID]
        if requireExercices!=[]:
            tempExo = set()
            tempData = tempData[(tempData['SUJET'].str.contains('|'.join(requireExercices), na=False))]
            for i in tempData['SUJET']:  # Loop to check if filtering the data have indeed all the exercices listed
                tempExo.add(i)
            if len(tempExo)==len(requireExercices):
                exerciceCheck = True
            else:
                continue
        if requireVersion!=[]:
            tempVersion = set()
            tempData = tempData[(tempData['VER'].astype(str).str.contains('|'.join(requireVersion), na=False))]
            for i in tempData['VER']:  # Loop to check if filtering the data have indeed all the exercices listed
                tempVersion.add(i)
            if len(tempVersion)==len(requireVersion):
                versionCheck = True
            else:
                continue
        if requireValidation:
            tempData2 = tempData[tempData.score==100]  # Filter temp data by keeping only exercices that have been validated
            if len(tempData2)<len(requireExercices):
                continue
            else:
                validationCheck = True
        if requireExperiment:
            tempData2 = tempData[tempData.EXP==1]
            if len(tempData2)<len(requireExercices):
                continue
            else:
                experimentCheck = True
        selectedLearners.append(learnerID)
    return selectedLearners

In [23]:
learners = selectLearners(data=particip_exercices,
                          learnerIDs = IDs,
                          requireExercices  = ["Melodie", "Alternance"],
                          requireVersion    = ["2", "3"],
                          requireValidation = False,
                          requireExperiment = False)

# mettre également les apprenants qui ne valident pas
# à verifier si les exercices sont faits en séquence (ordre : Mélodie, Alternance)
# Petit schéma d'extraction des données pour avoir le tableau global

In [24]:
len(learners)

632

In [25]:
particip_exercices[  (particip_exercices.LEARNER==learners[19])
                   & (particip_exercices['SUJET'].str.contains('|'.join(["Melodie","Alternance"])))
                   & (particip_exercices.VER==2)
                  ].reset_index()

,index,LEARNER,SUJET,VER,ERR,EXP,score,timestamp
0,1185,252,Melodie,2,Aucune,1,0,2021-06-09 07:47:29
1,1192,252,Melodie,2,Aucune,0,100,2021-06-09 07:47:38
2,1385,252,Alternance,2,Aucune,1,0,2021-06-09 07:52:21
3,1399,252,Alternance,2,Aucune,1,0,2021-06-09 07:52:33
4,1407,252,Alternance,2,Erreur de solution,0,0,2021-06-09 07:52:47
5,1417,252,Alternance,2,Erreur de solution,0,0,2021-06-09 07:53:02
6,1424,252,Alternance,2,Erreur de solution,0,0,2021-06-09 07:53:14
7,1451,252,Alternance,2,Erreur de solution,0,0,2021-06-09 07:53:50
8,1467,252,Alternance,2,Erreur de solution,0,0,2021-06-09 07:54:05
9,1483,252,Alternance,2,Aucune,0,100,2021-06-09 07:54:39


In [27]:
def expressExercisingMode(exerciceMode):
    if exerciceMode=="Validation sans erreur":
        return "valide sans erreur"
    elif exerciceMode=="Validation avec erreur":
        return "valide avec erreur"
    else:
        return "expérimente"

def getExercisingMode(experiment, score):
    if experiment==0 and score==100:
        return "Validation sans erreur"
    elif experiment==0 and score==0:
        return "Validation avec erreur"
    else:
        return "Experiment"
    
def getExercisingMode_(experiment, score):
    if experiment==0 and score==100:
        return "VAL+"
    elif experiment==0 and score==0:
        return "VAL-"
    else:
        return "EXP"

def getExerciceSkill(exoName, exoVersion):
    if exoName=="Melodie":
        if exoVersion==2:
            return ["C1*"]
        elif exoVersion==3 or exoVersion==4:
            return ["C2*"]
    elif exoName=="Alternance":
        if exoVersion==2 or exoVersion==3 or exoVersion==4:
            return ["C1","C2","C3*"]
    elif exoName=="Show Lumineux":
        if exoVersion==2 or exoVersion==3 or exoVersion==4:
            return ["C1","C2","C3*"]
    elif exoName=="Quelle Direction":
        if exoVersion==2 or exoVersion==3 or exoVersion==4:
            return ["C1","C2","C4","C6*"]
    elif exoName=="Instrument":
        if exoVersion==2 or exoVersion==3 or exoVersion==4:
            return ["C1","C2","C4","C6","C7*"]
    elif exoName=="Show Lumineux 2":
        if exoVersion==2 or exoVersion==3 or exoVersion==4:
            return ["C2","C5*","C6"]
    elif exoName=="Avertisseur":
        if exoVersion==2 or exoVersion==3 or exoVersion==4:
            return ["C1","C2","C3","C4","C6*","C7"]
    elif exoName=="Servo Chronometre":
        if exoVersion==2:
            return ["C1","C2","C3","C4","C6","C7*"]
        elif exoVersion==3 or exoVersion==4:
            return ["C2","C3","C4","C6","C7*"]
    elif exoName=="Sommeil Agreable":
        if exoVersion==2 or exoVersion==3 or exoVersion==4:
            return ["C2","C4","C6","C7*","C10"]
    elif exoName=="Reveil Matin":
        if exoVersion==2:
            return ["C1","C2","C4*","C9"]
        elif exoVersion==3 or exoVerion==4:
            return ["C1","C2","C4*","C9","C10"]
    elif exoName=="Secousses":
        if exoVersion==2:
            return ["C1","C2","C4*","C6","C7","C8","C9","C11*","C12"]
        elif exoVersion==3 or exoVersion==4:
            return ["C1","C2","C4*","C6","C7","C8","C9","C10","C11*","C12"]
    elif exoName=="Detection mouvement":
        if exoVersion==2:
            return ["C1","C2","C3","C4","C6","C7","C8","C9","C11*","C12"]
        elif exoVersion==3 or exoVersion==4:
            return ["C1","C2","C3","C4","C6","C7","C8","C9","C10","C11*","C12"]
    elif exoName=="Sirene":
        if exoVersion==2:
            return ["C2","C3","C4","C6","C7","C8","C9","C11*","C12"]
        elif exoVersion==3 or exoVersion==4:
            return ["C2","C3","C4","C6","C7","C8","C9","C10","C11*","C12"]
    elif exoName=="Applaudimetre":
        if exoVersion==2:
            return ["C1","C2","C4","C6","C7","C9","C10","C11*"]
        elif exoVersion==3:
            return ["C1","C2","C3","C6","C7","C9","C10","C11*"]
        elif exoVersion==4:
            return ["C1","C2","C3","C4","C6","C7","C8","C9","C10","C11*"]
    elif exoName=="Binaire":
        if exoVersion==2 or exoVersion==3 or exoVersion==4:
            return ["C1","C2","C3","C4","C6","C7","C8","C9","C10","C11*"]
    elif exoName=="Bille":
        if exoVersion==2 or exoVersion==3 or exoVersion==4:
            return ["C1","C2","C3","C5","C6","C7","C8","C9","C10","C11*"]
        
# returns a full exercicsing scenario for only one learner and one exercice
def scenarioSummary_Exercice(Learner):
    temp = Template("""
    ----------------------------------------------------------------------------
    t=$count,
    LearnerID: $learnerID,
    Connexion: $timestamp_co,
    Exercice:  $sujet,
    Version:   $sujetVer,
    Mode   :   $exMode,
    Error  :   $err,
    ----------------------------------------------------------------------------
    """)

# Automatically describe an exercising scenario for each learner (using learnerID), exercices, exercice version.
# data needs to be constrained for one learner, one exercice and one version. Filter the data before using scenarioDesc
def scenarioDesc(data):
    data = data.reset_index() # avoid any issues with indexations
    temp = Template("""
    * L'apprenant $learner commence l'exercice $exerciceName de version $exoVersion sur Quick-Pi;
    * Les compétences mobilisé dans l'exercice sont : $exoSkill;
    """)
    print(temp.substitute(learner=data["LEARNER"][0],
                    exerciceName=data["SUJET"][0], 
                    exoVersion=data["VER"][0],
                    exoSkill=getExerciceSkill(data["SUJET"][0], data["VER"][0])))
    temp2 = Template("""+ T=$count: à l'instant $timestamp_exo, l'apprenant $exerciceMode l'exercice $exerciceName;""")
    scenarioIter = str()
    scenario = str()
    dataSize = len(data)
    for timeStep in range(dataSize):
        print(temp2.substitute(
            count = timeStep,
            timestamp_exo=data["timestamp"][timeStep],
            learner=data["LEARNER"][timeStep],
            exerciceName=data["SUJET"][timeStep],
            exerciceMode=expressExercisingMode(getExercisingMode(data["EXP"][timeStep], data["score"][timeStep]))))
        scenarioIter += str(temp2)
    scenario = str(temp) + str(temp2)
    #return scenario

In [29]:
particip_exercices[
    (particip_exercices.LEARNER==learners[12])
    & (particip_exercices.SUJET=='Melodie')
    & (particip_exercices.VER==2)]

,LEARNER,SUJET,VER,ERR,EXP,score,timestamp
542,222,Melodie,2,Aucune,0,100,2021-06-09 07:31:07


In [33]:
 scenarioDesc(particip_exercices[
    (particip_exercices.LEARNER==learners[3])
    & (particip_exercices.SUJET=='Melodie')
    & (particip_exercices.VER==2)])


    * L'apprenant 176 commence l'exercice Melodie de version 2 sur Quick-Pi;
    * Les compétences mobilisé dans l'exercice sont : ['C1*'];
    
+ T=0: à l'instant 2021-06-09 07:06:59, l'apprenant expérimente l'exercice Melodie;
+ T=1: à l'instant 2021-06-09 07:07:53, l'apprenant expérimente l'exercice Melodie;
+ T=2: à l'instant 2021-06-09 07:07:55, l'apprenant expérimente l'exercice Melodie;
+ T=3: à l'instant 2021-06-09 07:07:59, l'apprenant expérimente l'exercice Melodie;
+ T=4: à l'instant 2021-06-09 07:08:00, l'apprenant expérimente l'exercice Melodie;
+ T=5: à l'instant 2021-06-09 07:08:02, l'apprenant expérimente l'exercice Melodie;


In [21]:
def genEvalData(data, learner, sujet, version):
    df = data[
    (data.LEARNER==learner)
    & (data.SUJET==sujet)
    & (data.VER==version)]
    df = df.reset_index()
    dataSize = len(df)
    result = [{'LEARNER' : learner, 
               'TIMESTEP': i,
               'EXERCICE': sujet,
               'SKILL':getExerciceSkill(sujet, version),
               'VALID/EXP':getExercisingMode_(df['EXP'][i], df['score'][i])} for i in range(dataSize)]
    df_r = pd.DataFrame(result)
    return df_r

In [22]:
df = genEvalData(data    = particip_exercices,
                 learner = learners[0],
                 sujet   = "Alternance",
                 version = 2
                )
df

IndexError: list index out of range

In [66]:
df.to_csv('EvalData/eval_data.csv')

In [67]:
def genAllEvalData(data, learnerPool, sujet, version):
    df   = pd.DataFrame()
    df_r = pd.DataFrame()
    for i in range(len(learnerPool)):
        df = genEvalData(data, learner = learnerPool[i], sujet=sujet, version=version)
        df_r = df_r.append(df, ignore_index=True)
        # df   = pd.DataFrame()
    return df_r     

In [68]:
df = genAllEvalData(data    = particip_exercices,
                 learnerPool = learners,
                 sujet   = "Alternance",
                 version = 2
                )
df

,LEARNER,TIMESTEP,EXERCICE,SKILL,VALID/EXP
0,162,0,Alternance,"[C1, C2, C3*]",VAL-
1,162,1,Alternance,"[C1, C2, C3*]",VAL+
2,163,0,Alternance,"[C1, C2, C3*]",EXP
3,163,1,Alternance,"[C1, C2, C3*]",EXP
4,163,2,Alternance,"[C1, C2, C3*]",EXP
...,...,...,...,...,...
6250,4674,0,Alternance,"[C1, C2, C3*]",EXP
6251,4674,1,Alternance,"[C1, C2, C3*]",VAL-
6252,4674,2,Alternance,"[C1, C2, C3*]",VAL+
6253,4692,0,Alternance,"[C1, C2, C3*]",EXP


In [69]:
df.to_csv('EvalData/eval_data.csv')

In [70]:
# All of the below functions will serve us to create learner Profile.

In [1]:
 scenarioDesc(particip_exercices[
    (particip_exercices.LEARNER==learners[0])
    & (particip_exercices.SUJET=='')
    & (particip_exercices.VER==2)])

NameError: name 'scenarioDesc' is not defined